In [27]:
# import pandas as pd
# t = pd.read_csv('./train.csv')
# t.drop(columns=['id'], inplace=True)

In [28]:
# train = t[:14000]
# valid = t[19000:]
# test = t[19000:]

In [29]:
# train.to_csv('./15kTrain.csv', index=False)
# valid.to_csv('./5kValid.csv', index=False)
# test.to_csv('./test.csv', index=False)

In [30]:
cd .. 

/Users/chenmargalit/Desktop/Python Playground


In [31]:
cd spooky-author-identification/

/Users/chenmargalit/Desktop/Python Playground/spooky-author-identification


In [32]:
!ls

First model - Fasttext.ipynb test579.csv
Untitled.ipynb               train.csv
Untitled1.ipynb              train15k.csv
test.csv                     validation4k.csv


In [33]:
import pandas as pd
t = pd.read_csv('./train15k.csv')

In [34]:
import torch
from torchtext import data, datasets
from torchtext.data import TabularDataset
import random

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField()

In [35]:
%%time
train, val = TabularDataset.splits(
path = './', format='csv', skip_header=True, train = './train15k.csv', validation = './validation4k.csv',
fields = [('text', TEXT), ('author', LABEL)]
)

CPU times: user 17.7 s, sys: 522 ms, total: 18.3 s
Wall time: 18.5 s


In [36]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train)

In [53]:
len(LABEL.vocab)

3

In [37]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x1227230d0>, {'EAP': 0, 'MWS': 1, 'HPL': 2})


In [38]:
%%time
BATCH_SIZE = 64
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train, val), 
    batch_size = BATCH_SIZE, 
    sort_key = lambda x: len(x.text),    
    device = device)

CPU times: user 131 µs, sys: 20 µs, total: 151 µs
Wall time: 158 µs


In [39]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        print('convs', self.convs)
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [40]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

convs ModuleList(
  (0): Conv2d(1, 100, kernel_size=(2, 100), stride=(1, 1))
  (1): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
  (2): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
)


In [41]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,575,403 trainable parameters


In [42]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.1077,  0.1105,  0.5981,  ..., -0.8316,  0.4529,  0.0826],
        ...,
        [ 0.3800, -0.4023, -0.1590,  ..., -0.2469, -0.2327,  0.0116],
        [ 1.1719, -1.7597, -0.2668,  ..., -2.4071, -0.0571,  1.5848],
        [ 0.2093,  1.6408, -0.4784,  ..., -0.0414,  0.4682, -0.8824]])

In [43]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [44]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [45]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [46]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)
        
        loss = criterion(predictions, batch.author)
        
        acc = categorical_accuracy(predictions, batch.author)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [47]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.author)
            
            acc = categorical_accuracy(predictions, batch.author)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [48]:
%%time

N_EPOCHS = 3

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

	Train Loss: 0.907 | Train Acc: 56.39%
	 Val. Loss: 0.713 |  Val. Acc: 70.31%
	Train Loss: 0.620 | Train Acc: 74.65%
	 Val. Loss: 0.561 |  Val. Acc: 77.75%
	Train Loss: 0.432 | Train Acc: 83.08%
	 Val. Loss: 0.501 |  Val. Acc: 80.33%
CPU times: user 2min 51s, sys: 14.8 s, total: 3min 5s
Wall time: 3min 17s


In [49]:
import spacy
nlp = spacy.load('en')

def predict_class(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [50]:
pred_class = predict_class(model, "I had to tell Harris because he owned the house and deserved to know what had gone out of it.")
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

Predicted class is: 2 = HPL


In [51]:
test.head(2)

NameError: name 'test' is not defined

In [ ]:
def d(num):
    return test.text[(19_000 + num)], test.author[(19_000 + num)]    

In [ ]:
len(test)

In [ ]:
d(102)